<a href="https://colab.research.google.com/github/8Dis-like/UCLALearning/blob/main/IBM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E213 Module 2 – Live Session: Data Acquisition & Financial Ratios (Alpha Vantage)

In this notebook, we will:
1. Load a secret API key from a `.env` file (so we don't hard-code secrets).
2. Call the **Alpha Vantage** API to get real company financials.
3. Convert JSON (raw API data) into Pandas DataFrames.
4. Clean numbers and align tables by date.
5. Compute key ratios: **Current Ratio**, **Debt-to-Equity**, **Gross/Operating/Net Margins**, **ROA**, **ROE**.

## 0) One-time Setup (do before running the rest)
1. Install packages (in a terminal):
   ```bash
   pip install pandas requests python-dotenv
   ```
2. Create a file named **`.env`** in the **same folder** as this notebook with this content:
   ```
   ALPHAVANTAGE_API_KEY=YOUR_KEY_HERE
   ```
3. Get a free key from Alpha Vantage: https://www.alphavantage.co/support/#api-key

**Why `.env`?** It keeps secrets out of your code and out of Canvas submissions.

## 1) Imports & Loading the API Key

In [ ]:
import os              # for reading environment variables (for example: API key)
import requests        # for web APIs (HTTP) calls
import pandas as pd    # data structure +
from dotenv import load_dotenv  # get environment variables from .env file

# .env file must be in the current (Jupyter-Notebook home) folder
load_dotenv()

# Get API key
API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

# optional: missing key error handling
if not API_KEY:
    raise SystemExit("ALPHAVANTAGE_API_KEY not found. Create a .env file with ALPHAVANTAGE_API_KEY=YOUR_KEY_HERE")

# AAPL, MSFT, AMZN, etc. - stocker ticker
SYMBOL = "IBM"

# Base URL for Alpha Vantage.
BASE = "https://www.alphavantage.co/query"


## 2) A tiny helper function to call the API safely given the free API key limits
- Here is a function created for this class to make API calls.
- It adds the function name (like `INCOME_STATEMENT`) and our symbol.
- It also checks for common API issues:
  - **Rate limit** (too many requests/minute) → shows a helpful message.
  - **Error Message** (bad parameters) → shows a helpful message.

In [ ]:
def av_fetch(function, symbol):
    """
    Fetch data from Alpha Vantage.

    Arguments:
        function -- the API function name (e.g., "TIME_SERIES_DAILY")
        symbol -- the stock ticker symbol (e.g., "AAPL")

    Returns:
        A Python dictionary created from the JSON data returned by Alpha Vantage.
    """

    # Create a dictionary of parameters for the web request.
    # These key-value pairs will be sent to Alpha Vantage.
    params = {
        "function": function,
        "symbol": symbol,
        "apikey": API_KEY
    }

    # Make a web request to Alpha Vantage using the base URL and our parameters.
    response = requests.get(BASE, params=params, timeout=20)

    # If Alpha Vantage returns a non-OK (not 200) HTTP status code, raise an error.
    response.raise_for_status()

    # Convert the JSON response text into a Python dictionary or list.
    data = response.json()

    # If we hit the free-tier limit, Alpha Vantage includes a "Note" in the data.
    # Example:
    # "Note": "Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day."
    if isinstance(data, dict) and "Note" in data:
        raise RuntimeError("Rate limit hit. Wait ~60 seconds and run the cell again.")

    # If the function or symbol is wrong, Alpha Vantage includes an "Error Message".
    if isinstance(data, dict) and "Error Message" in data:
        raise RuntimeError(f"API error: {data['Error Message']}")

    # Return the Python dictionary to the caller.
    return data


## 3) Download **Income Statement** and **Balance Sheet**
**Annual Data**. Alpha Vantage returns JSON with keys like `annualReports` and `quarterlyReports`.

In [ ]:
# 3A) Income Statement
inc = av_fetch("INCOME_STATEMENT", SYMBOL)
                    # use dict.get(key, default) to retrieve data -- Alternatively inc["annualReports"] but not safe.
inc_df = pd.DataFrame(inc.get("annualReports", []))
print("Income statement columns:")
print(list(inc_df.columns))
inc_df.head(5)

Income statement columns:
['fiscalDateEnding', 'reportedCurrency', 'grossProfit', 'totalRevenue', 'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome', 'sellingGeneralAndAdministrative', 'researchAndDevelopment', 'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome', 'interestIncome', 'interestExpense', 'nonInterestIncome', 'otherNonOperatingIncome', 'depreciation', 'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense', 'interestAndDebtExpense', 'netIncomeFromContinuingOperations', 'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda', 'netIncome']


,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,2024-12-31,USD,35551000000,62753000000,27201000000,27201000000,10074000000,16737000000,7479000000,25478000000,...,None,4667000000,5797000000,-218000000,None,6015000000,None,7509000000,12176000000,6023000000
1,2023-12-31,USD,34300000000,61860000000,27560000000,27560000000,7514000000,17952000000,6631000000,26786000000,...,None,4395000000,8690000000,1176000000,None,7099000000,None,7514000000,7514000000,7502000000
2,2022-12-31,USD,32687000000,60530000000,27842000000,27842000000,8174000000,16103000000,6567000000,24514000000,...,None,4802000000,1156000000,-626000000,None,1782000000,None,2372000000,7174000000,1640000000
3,2021-12-31,USD,31486000000,57351000000,25865000000,25865000000,6832000000,17699000000,6488000000,24654000000,...,None,6417000000,4837000000,124000000,None,4712000000,None,7021000000,13438000000,5742000000
4,2020-12-31,USD,35575000000,73621000000,38046000000,38046000000,6895000000,20308000000,6333000000,28680000000,...,None,6695000000,4637000000,-864000000,None,5501000000,None,6014000000,12709000000,5590000000


In [ ]:
# 3B) Balance Sheet
bal = av_fetch("BALANCE_SHEET", SYMBOL)
bal_df = pd.DataFrame(bal.get("annualReports", []))
print("Balance sheet columns:")
print(list(bal_df.columns))
bal_df.head(5)

Balance sheet columns:
['fiscalDateEnding', 'reportedCurrency', 'totalAssets', 'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue', 'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables', 'totalNonCurrentAssets', 'propertyPlantEquipment', 'accumulatedDepreciationAmortizationPPE', 'intangibleAssets', 'intangibleAssetsExcludingGoodwill', 'goodwill', 'investments', 'longTermInvestments', 'shortTermInvestments', 'otherCurrentAssets', 'otherNonCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities', 'currentAccountsPayable', 'deferredRevenue', 'currentDebt', 'shortTermDebt', 'totalNonCurrentLiabilities', 'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt', 'longTermDebtNoncurrent', 'shortLongTermDebtTotal', 'otherCurrentLiabilities', 'otherNonCurrentLiabilities', 'totalShareholderEquity', 'treasuryStock', 'retainedEarnings', 'commonStock', 'commonStockSharesOutstanding']


,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2024-12-31,USD,137175000000,34482000000,13947000000,13947000000,1289000000,14010000000,102694000000,8928000000,...,5089000000,None,58396000000,7313000000,981000000,27307000000,None,151163000000,61380000000,937200000
1,2023-12-31,USD,135241000000,32908000000,13068000000,13068000000,1161000000,13956000000,102333000000,8721000000,...,6426000000,None,59935000000,7023000000,1164000000,22533000000,None,151276000000,59643000000,922073828
2,2022-12-31,USD,127243000000,29118000000,7886000000,7886000000,1552000000,7005000000,98126000000,8212000000,...,4760000000,None,54013000000,7592000000,12243000000,21944000000,None,149825000000,58343000000,912269062
3,2021-12-31,USD,132001000000,29539000000,6650000000,6650000000,1649000000,14977000000,102460000000,8916000000,...,6787000000,None,55139000000,9386000000,13995000000,18901000000,-169392000000,154209000000,57319000000,904641001
4,2020-12-31,USD,155970000000,39165000000,13212000000,13212000000,1839000000,18738000000,116805000000,10040000000,...,7183000000,None,62895000000,13588000000,36719000000,20597000000,-169339000000,162717000000,56556000000,896600000


## 4) Clean & Merge Data
This cleans the data and selects ONLY the columns we need to calculate the ratios.

- From the **income statement**:
  - `totalRevenue`, `costOfRevenue`
- From the **balance sheet**:
  - `totalAssets`, `inventory`

`fiscalDateEnding` (the period label) is also needed.

In [ ]:
# Select only the columns needed for the specific ratios
# Needed: totalRevenue, costOfRevenue, totalAssets, inventory
cols_inc = ["fiscalDateEnding", "totalRevenue", "costOfRevenue"]
cols_bal = ["fiscalDateEnding", "totalAssets", "inventory"]

# Create clean subsets
inc_clean = inc_df[cols_inc].copy()
bal_clean = bal_df[cols_bal].copy()

# Convert columns to numeric (coercing errors to NaN)
for col in ["totalRevenue", "costOfRevenue"]:
    inc_clean[col] = pd.to_numeric(inc_clean[col], errors="coerce")

for col in ["totalAssets", "inventory"]:
    bal_clean[col] = pd.to_numeric(bal_clean[col], errors="coerce")

# Set Date as Index to join them easily
inc_clean.set_index("fiscalDateEnding", inplace=True)
bal_clean.set_index("fiscalDateEnding", inplace=True)

# Merge them into one table
financials = inc_clean.join(bal_clean, how="inner")

# Convert the index to Datetime objects so we can filter by Year easily
financials.index = pd.to_datetime(financials.index)

# Show the cleaned table
print("Merged Financial Data:")
display(financials.head())

Merged Financial Data:


,totalRevenue,costOfRevenue,totalAssets,inventory
fiscalDateEnding,,,,
2024-12-31,62753000000,27201000000,137175000000,1289000000
2023-12-31,61860000000,27560000000,135241000000,1161000000
2022-12-31,60530000000,27842000000,127243000000,1552000000
2021-12-31,57351000000,25865000000,132001000000,1649000000
2020-12-31,73621000000,38046000000,155970000000,1839000000


## 5) Calculate and Compare Ratios for 2022 & 2023
This implements the specific formulas to calculate `Total Asset Turnover` and `Inventory Turnover`.

In [ ]:
# Filter for only the years 2022 and 2023
# We look for rows where the year is 2022 or 2023
target_years = [2023, 2022]
analysis_df = financials[financials.index.year.isin(target_years)].copy()

# --- Calculate Ratios ---

# 1. Total Asset Turnover = Total Revenue / Total Assets
analysis_df["Total Asset Turnover"] = analysis_df["totalRevenue"] / analysis_df["totalAssets"]

# 2. Inventory Turnover = Cost of Revenue / Inventory
# Note: Sometimes Alpha Vantage uses "costofGoodsAndServicesSold" but usually "costOfRevenue" covers it.
analysis_df["Inventory Turnover"] = analysis_df["costOfRevenue"] / analysis_df["inventory"]

# Display the results for comparison
print(f"=== {SYMBOL} Financial Ratios Comparison (2023 vs 2022) ===")
display(analysis_df[["Total Asset Turnover", "Inventory Turnover"]])

=== IBM Financial Ratios Comparison (2023 vs 2022) ===


,Total Asset Turnover,Inventory Turnover
fiscalDateEnding,,
2023-12-31,0.457406,23.738157
2022-12-31,0.475704,17.939433


## 6) Observations & Interpretation

### 1. Total Asset Turnover
* **2023:** 0.457406
* **2022:** 0.475704
* **Analysis:** Comparing 2023 to 2022, IBM's efficiency in using its assets worsened. A higher ratio indicates the company is generating more revenue for every dollar of assets it owns. This indicates that IBM's efficiency in utilizing its total asset base to generate revenue has worsened over the past year. Since a higher ratio suggests the company is generating more revenue for every dollar of assets it owns, this decline may suggest that the growth of IBM's asset base outpaced its revenue growth during this period.

### 2. Inventory Turnover
* **2023:** 23.738157
* **2022:** 17.939433
* **Analysis:** The inventory turnover has increased. A higher inventory turnover generally indicates better sales performance or more efficient inventory management (less stock sitting idle). This represents a substantial improvement in operational efficiency regarding inventory management. A higher turnover ratio generally points to better sales performance or a more streamlined supply chain where less stock remains idle

### Conclusion
Based on these results, IBM’s financial health and operational efficiency in 2023 present a mixed but generally positive trend in specific operational areas:

• Operational Efficiency: While overall asset utilization efficiency dipped slightly, IBM demonstrated stronger lean management of its physical goods. The jump in inventory turnover suggests that IBM is moving products more quickly or has successfully reduced the costs associated with holding excess inventory. In the context of a "Business Transformation" focus, this aligns with IBM’s goals of reducing costs and increasing effectiveness.


• Financial Health: The improved inventory turnover indicates robust sales performance and high demand for IBM's offerings in 2023 compared to 2022. However, the declining Total Asset Turnover suggests that the company may need to focus on better leveraging its non-inventory assets—such as its massive investments in cloud infrastructure and AI platforms (like watsonx and IBM Cloud)—to drive higher revenue returns.


Overall Verdict: IBM's operational efficiency in 2023 appears stronger in terms of inventory management, indicating a more agile response to market demand, even as it works to optimize the revenue-generating potential of its broader asset base.